In [170]:
!pip install noisereduce

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
import noisereduce as nr
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [180]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your spectrogram images
train_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/train'
validation_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

#train_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/train'
#validation_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

#reserved_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/reserved'

batchSize = 1

# Image dimensions
#img_width, img_height = 884, 322
img_width, img_height = 322, 884
# Rescale the images by dividing every pixel in every image by 255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
reserved_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directory and prepare them for training
train_generator = train_datagen.flow_from_directory(
    #train_directory,
    directory = train_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 91825)

validation_generator = validation_datagen.flow_from_directory(
    #validation_directory,
    directory = validation_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 18232)

#reserved_generator = reserved_datagen.flow_from_directory(
#    directory = reserved_directory,
#    target_size=(img_height, img_width),
#    color_mode = "grayscale",
#    batch_size=16,
#    class_mode='categorical',
#    shuffle = False)

Found 1836 images belonging to 10 classes.
Found 572 images belonging to 10 classes.


In [181]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow import keras
# Define the CNN model
model = Sequential()

# Convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(322, 884, 1) ) )
model.add(MaxPooling2D((2, 2)))

# Adding another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((5, 5)))

#model.add(Conv2D(64, (5, 5), activation='relu'))
#model.add(MaxPooling2D((4, 4)))

#Adding dropout layer to control overfitting
model.add(Dropout(0.25))
# Adding dense softmax layer
model.add(Dense(128, activation='softmax'))

# Adding layer with tanh acitvation function to improve accuracy
model.add(Conv2D(128, (3, 3), activation='tanh'))
#varying the size of the max pooling kernel has helepd the model
model.add(MaxPooling2D((4, 4)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

#several layers added


# Dropout for regularization
model.add(Dropout(0.15))

# Output layer with a unit for each class and softmax activation
model.add(Dense(10, activation='softmax'))

# Compile the model

#opti = keras.optimizers.Adam(learning_rate = 0.02)
model.compile( optimizer = 'adam',#optimizer = opti,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_140 (Conv2D)         (None, 320, 882, 32)      320       
                                                                 
 max_pooling2d_134 (MaxPool  (None, 160, 441, 32)      0         
 ing2D)                                                          
                                                                 
 conv2d_141 (Conv2D)         (None, 158, 439, 64)      18496     
                                                                 
 max_pooling2d_135 (MaxPool  (None, 31, 87, 64)        0         
 ing2D)                                                          
                                                                 
 dropout_67 (Dropout)        (None, 31, 87, 64)        0         
                                                                 
 dense_97 (Dense)            (None, 31, 87, 128)     

In [183]:
# Train the model

fitModel = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batchSize,
    epochs = 40,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batchSize)


Epoch 1/40
1836/1836 [==============================] - 43s 24ms/step - loss: 0.7321 - accuracy: 0.7614 - val_loss: 2.0779 - val_accuracy: 0.6748
Epoch 2/40
1836/1836 [==============================] - 38s 21ms/step - loss: 0.6789 - accuracy: 0.7636 - val_loss: 1.5890 - val_accuracy: 0.6591
Epoch 3/40
1836/1836 [==============================] - 43s 24ms/step - loss: 0.7206 - accuracy: 0.7533 - val_loss: 1.5439 - val_accuracy: 0.7080
Epoch 4/40
1836/1836 [==============================] - 43s 23ms/step - loss: 0.6189 - accuracy: 0.7865 - val_loss: 1.6112 - val_accuracy: 0.6853
Epoch 5/40
1836/1836 [==============================] - 43s 23ms/step - loss: 0.6274 - accuracy: 0.7756 - val_loss: 1.6502 - val_accuracy: 0.6871
Epoch 6/40
1836/1836 [==============================] - 38s 21ms/step - loss: 0.6208 - accuracy: 0.7827 - val_loss: 1.8179 - val_accuracy: 0.6661
Epoch 7/40
1836/1836 [==============================] - 43s 24ms/step - loss: 0.6272 - accuracy: 0.7821 - val_loss: 1.8128 -

In [184]:
test_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

test_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.9625)

test_generator = test_datagen.flow_from_directory(
    directory = test_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size= batchSize,
    class_mode='categorical',
    subset = 'validation',
    shuffle = False) #since it's test, don't set shuffle to true


Found 544 images belonging to 10 classes.


In [185]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batchSize)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


544/544 [==============================] - 6s 11ms/step - loss: 3.2186 - accuracy: 0.6710
Test accuracy: 0.6709558963775635
Test loss: 3.2186334133148193


In [186]:
from sklearn.metrics import classification_report, confusion_matrix
import random
# # Predict classes
#predictions = model.predict(test_generator, steps=test_generator.samples // 32)
predictions = model.predict(test_generator, steps=test_generator.samples // batchSize)
predicted_classes = np.argmax(predictions, axis=1)

# # Get true labels
#true_classes = random.sample(test_generator.classes, 544)
true_classes = test_generator.classes
#scores report
report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys(), zero_division = 0)

print(report)


544/544 [==============================] - 7s 13ms/step
              precision    recall  f1-score   support

     afbfly1       0.07      0.18      0.10        28
     afdfly1       0.88      0.78      0.82        36
     chespa1       0.00      0.00      0.00         1
     chewea1       1.00      0.44      0.62         9
     grywrw1       0.88      0.59      0.71        49
     luebus1       0.68      0.68      0.68        31
     nobfly1       0.62      0.14      0.23        35
     rocmar2       0.20      0.35      0.25        26
     somgre1       0.86      0.85      0.86       304
     somtit4       0.35      0.24      0.29        25

    accuracy                           0.67       544
   macro avg       0.55      0.42      0.45       544
weighted avg       0.74      0.67      0.69       544



In [187]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)


[[  5   0   0   0   0   0   0  16   6   1]
 [  3  28   0   0   0   0   0   3   1   1]
 [  0   0   0   0   0   0   0   0   1   0]
 [  2   0   0   4   0   0   1   0   2   0]
 [  4   0   0   0  29   2   0   0  12   2]
 [  2   0   0   0   0  21   0   0   8   0]
 [ 15   1   0   0   0   0   5   8   4   2]
 [ 12   2   0   0   0   0   2   9   1   0]
 [ 20   1   0   0   4   7   0   9 258   5]
 [ 12   0   0   0   0   1   0   0   6   6]]
